# Pydna / Biopython behaviour

Here some code chunks showing some important points from biopython / pydna

## How to know if a sequence is circular or linear

This is important, and might not be straightforward from certain formats.

In [40]:
from pydna.parsers import parse as pydna_parse
from Bio.SeqIO import read as seqio_read
from pydna.dseqrecord import Dseqrecord
# Pydna reader gets it right from the genbank file:
dseqrecords = pydna_parse('../plasmids/addgene-plasmid-39296-sequence-49545.gbk')
print('pydna with gb:',dseqrecords[0].circular)

seqrecord = seqio_read('../plasmids/addgene-plasmid-39296-sequence-49545.dna','snapgene')

# Simple conversion will not get right the fact that it is circular
dseqrecord = Dseqrecord(seqrecord)
print('seqio with snapgene, then convert to dseqrecord:',dseqrecord.circular)

# Check in the properties to tell if the topology is circular
iscircular = 'topology' in seqrecord.annotations.keys() and seqrecord.annotations['topology'] == 'circular'
dseqrecord = Dseqrecord(seqrecord, circular=iscircular)
print('seqio with snapgene, then convert to dseqrecord:',dseqrecord.circular)


pydna with gb: True
seqio with snapgene, then convert to dseqrecord: False
seqio with snapgene, then convert to dseqrecord: True


## Restriction enzymes

In [58]:
# List of commercial enzymes
from Bio.Restriction.Restriction import CommOnly
from pydna.dseqrecord import Dseqrecord

enzyme = CommOnly.format('EcoRI')

# If we want to know where the enzymes cut in the original sequence
# For linear fragments it's easy, just need to know the boundaries
# We know that EcoRI should cut after the G here
seq = 'AAAAAAGAATTCTTTTTTTT'
dseq = Dseqrecord(seq,circular=False)

cutted_list = dseq.cut([enzyme])
print('linear')

for i in cutted_list:
    print(len(i),i.seq)
    
# What about circular sequences?
seq = 'AAAAAAGAATTCTTTTTTAAAGAATTCCCCCCC'
dseq = Dseqrecord(seq,circular=True)
cutted_list = dseq.seq.cut([enzyme])

for i in cutted_list:
    print(i.pos)
    print(type(i))

linear
11 AAAAAAGAATT
13 AATTCTTTTTTTT
22
<class 'pydna.dseq.Dseq'>
40
<class 'pydna.dseq.Dseq'>
